<a href="https://colab.research.google.com/github/vivek09thakur/PANDA/blob/main/Colab%20Notebook/Panda_Code_Refactored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Refactored Code of PANDA**

- [x] **DRIVE MOUNTED**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- [x] **A Refactored version Paradigm based Artificial Neural Dailogue Agent (P.A.N.D.A)**

In [21]:
import os
import sys
import time
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


class PANDA:

    def __init__(self,prompts,model_name,tokens=25):
        self.prompts = prompts
        self.tokens = tokens
        with open(self.prompts,'r') as f:
            # Read the lines from the prompts file
            self.text_data = f.readlines()
        # Create a tokenizer
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.text_data)
        self.total_words = len(self.tokenizer.word_index) + 1
        self.model_name = model_name
        # self.max_sequence_len = 0


    def preprocess_data(self):
        self.input_sequences = []
        self.next_words = []
        for line in self.text_data:
            # Convert the text to sequences
            token_list = self.tokenizer.texts_to_sequences([line])[0]
            for i in range(1,len(token_list)):
                # Create n-grams
                n_grams = token_list[:i+1]
                self.input_sequences.append(n_grams)
                self.next_words.append(token_list[i])

    def generate_pad_sequences(self):
        # Pad sequences
        self.max_sequence_len = max([len(x) for x in self.input_sequences])
        self.input_sequences = np.array(
            pad_sequences(self.input_sequences,
                          maxlen=self.max_sequence_len,padding='pre'))
        self.predictors, self.label = self.input_sequences[:, :-1], self.input_sequences[:, -1]

    def create_model(self):
        # Create model
        self.model = Sequential()
        self.model.add(Embedding(self.total_words,100,
                                 input_length=self.max_sequence_len-1))
        self.model.add(LSTM(500))
        self.model.add(Dense(self.total_words,activation='softmax'))
        self.model.compile(loss='sparse_categorical_crossentropy',
                           optimizer='adam',metrics=['accuracy'])
        self.model.fit(self.predictors,self.label,epochs=500,
                       verbose=1)
        self.model.save(self.model_name)

    def load_model(self):
        self.model = load_model(self.model_name)

    def train_or_load_model(self):
        if os.path.exists(self.model_name):
            self.load_model()
        else:
            self.create_model()

    def completion(self,user_input):
        # Predictions
        token_list = self.tokenizer.texts_to_sequences([user_input])[0]
        token_list = pad_sequences([token_list],
                                      maxlen=self.max_sequence_len-1,
                                      padding='pre')
        predicted_words = []

        for _ in range(self.tokens):
            predicted = self.model.predict(token_list,verbose=0)
            predicted_index = np.argmax(predicted)
            output_word = ''
            for word,index in self.tokenizer.word_index.items():
                if index == predicted_index:
                    output_word = word
                    break
            predicted_words.append(output_word)
            token_list = np.append(
                token_list[:,1:],
                [[predicted_index]],
                axis=1)
        return ' '.join(predicted_words)

    def type_response(self,response):
        for char in response:
            sys.stdout.write(char)
            sys.stdout.flush()
            time.sleep(0.01)
        print()

    def introduce(self):
        print('Hello, I am PANDA, Paradgim-based Artificial Neural Dialogue Agent. An AI Language Model which is able to predict next sequence of words based on the input sequence of words.')


- [ ] **Test Run**

In [22]:
parameters = [
    '/content/prompt_completion.txt', # prompts file
    'Saved Model/panda.h5', # model name
     25 # number of tokens to generate
]

panda = PANDA(parameters[0],parameters[1],parameters[2])
panda.preprocess_data()
panda.generate_pad_sequences()

if __name__=='__main__':

    panda.train_or_load_model()
    panda.introduce()

    while True:
        prompts = input(f"\n ↳ (user) : " )
        completion = panda.completion(prompts)
        panda.type_response(completion)

Epoch 1/500
55/55 [==============================] - 10s 145ms/step - loss: 5.4415 - accuracy: 0.0308
Epoch 2/500
55/55 [==============================] - 3s 46ms/step - loss: 4.9110 - accuracy: 0.0718
Epoch 3/500
55/55 [==============================] - 2s 42ms/step - loss: 4.2220 - accuracy: 0.1231
Epoch 4/500
55/55 [==============================] - 1s 19ms/step - loss: 3.6151 - accuracy: 0.2177
Epoch 5/500
55/55 [==============================] - 1s 24ms/step - loss: 3.0743 - accuracy: 0.3276
Epoch 6/500
55/55 [==============================] - 1s 18ms/step - loss: 2.6353 - accuracy: 0.4011
Epoch 7/500
55/55 [==============================] - 1s 16ms/step - loss: 2.2258 - accuracy: 0.4604
Epoch 8/500
55/55 [==============================] - 1s 19ms/step - loss: 1.8954 - accuracy: 0.5276
Epoch 9/500
55/55 [==============================] - 1s 19ms/step - loss: 1.6051 - accuracy: 0.5869
Epoch 10/500
55/55 [==============================] - 1s 23ms/step - loss: 1.3475 - accuracy: 0.63

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Hello, I am PANDA, Paradgim-based Artificial Neural Dialogue Agent. An AI Language Model which is able to predict next sequence of words based on the input sequence of words.

 ↳ (user) : What is your name buddy?
my name is panda which stands for paradigm based artificial neural dialogue agent agent agent believe it's to believe while others believe it's to fulfill

 ↳ (user) : Write a code for hello world program in c
language include stdio h int main printf hello world return 0 printf enter a three digit integer scanf d n originaln n while originaln 0

 ↳ (user) : Write a code of hello world in c language.
include stdio h int main printf hello world return 0 printf enter a three digit integer scanf d n originaln n while originaln 0 remainder


KeyboardInterrupt: ignored